In [19]:
rstate=10

In [20]:
import string 
import seaborn as sns
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
import warnings
warnings.filterwarnings("ignore")
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd
import io

In [21]:
df = pd.read_csv(r"G:\OneDrive - northsouth.edu\CODES\PROJECTS\PROJECT-CSE299-CYBERBULLYING PREVENTION-DETECTION and MENTAL COUNSELING SYSTEM\Machine Learning Models\Bengali hate speech.csv")
df.sample(frac=1)

,sentence,hate,category
6446,চার নাস্তিক কুত্তার বাচ্চাদেরকে জুতা পেটা করতা...,1,religion
13521,দারুন গল্প,0,entertainment
19727,তাহসিনেশনের নকল পুরাদমে,0,religion
5491,ফরজের খবর নাই ছুন্নাত চোদায় ভন্ড মাদার চোদ,1,religion
29175,সালমানের চার বাগের এক ঘাওহবে না এখন কার নায়ক ক...,0,"Meme, TikTok and others"
...,...,...,...
13768,নাটকটা অনেক ভাল লাগলো,0,entertainment
17601,পুলিশের কাজে বাঁধা দেওয়ার কারণে এদেরকে জরিমানা...,0,crime
20285,আসসালামু আলাইকুম কেমন আছেন সিলেট হবিগঞ্জ থেকে ...,0,religion
8017,খাংকির পোলা মোনাজাত ধর,1,celebrity


In [22]:
import string

In [23]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
def remove_punctuation_from_sentence(sentence):
    sentence_without_punctuation="".join([c for c in sentence if c not in string.punctuation])
    return sentence_without_punctuation

In [25]:
df['sentence without punctuation'] = df['sentence'].apply(lambda x:remove_punctuation_from_sentence(x))
df.head()

,sentence,hate,category,sentence without punctuation
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব


In [26]:
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bnlp.corpus.util import remove_stopwords

In [27]:
basic_tokenizer = BasicTokenizer()
df['tokenized_sentence'] = df['sentence without punctuation'].apply(lambda x: basic_tokenizer.tokenize(x))
df.head()

,sentence,hate,category,sentence without punctuation,tokenized_sentence
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী,"[যত্তসব, পাপন, শালার, ফাজলামী]"
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার,"[পাপন, শালা, রে, রিমান্ডে, নেওয়া, দরকার]"
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,"[জিল্লুর, রহমান, স্যারের, ছেলে, এতো, বড়, জারজ,..."
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,"[শালা, লুচ্চা, দেখতে, পাঠার, মত, দেখা, যায়]"
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,"[তুই, তো, শালা, গাজা, খাইছচ, ।, তুর, মার, হেডা..."


In [28]:
df['Removed Stopped word'] = df['sentence without punctuation'].apply(lambda x: remove_stopwords(x,stopwords))
df.head()

,sentence,hate,category,sentence without punctuation,tokenized_sentence,Removed Stopped word
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী,"[যত্তসব, পাপন, শালার, ফাজলামী]","[যত্তসব, পাপন, শালার, ফাজলামী]"
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার,"[পাপন, শালা, রে, রিমান্ডে, নেওয়া, দরকার]","[পাপন, শালা, রে, রিমান্ডে, দরকার]"
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,"[জিল্লুর, রহমান, স্যারের, ছেলে, এতো, বড়, জারজ,...","[জিল্লুর, রহমান, স্যারের, ছেলে, এতো, বড়, জারজ,..."
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,"[শালা, লুচ্চা, দেখতে, পাঠার, মত, দেখা, যায়]","[শালা, লুচ্চা, পাঠার, মত, যায়]"
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,"[তুই, তো, শালা, গাজা, খাইছচ, ।, তুর, মার, হেডা...","[তুই, শালা, গাজা, খাইছচ, ।, তুর, মার, হেডায়, খ..."


In [29]:
df['hate_speech']=df['Removed Stopped word'].apply(' '.join)

In [30]:
x=df['hate_speech']
y=df['hate']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=rstate)

In [32]:
cv = CountVectorizer(stop_words = stopwords)
cv.fit(x)

# transform the training and testing data using count vectorizer object
X_train_count =  cv.transform(X_train)
X_test_count =  cv.transform(X_test)

#Tfid transformer
tfidf_tr = TfidfTransformer()
X_train_count_tfidf = tfidf_tr.fit_transform(X_train_count)
X_test_count_tfidf = tfidf_tr.transform(X_test_count)

# ngram level tf-idf 
tv_ngram = TfidfVectorizer(ngram_range=(1,2))
tv_ngram.fit(x)
X_train_tfidf_ngram =  tv_ngram.transform(X_train)
X_test_tfidf_ngram =  tv_ngram.transform(X_test)

In [33]:
def classifierModel(classifier, Xtrain, Ytrain, Xtest,roc):
    classifier.fit(Xtrain, Ytrain)
    predictions = classifier.predict(Xtest)
    print(30*'=')
    print(f"Accuracy : {accuracy_score(predictions, y_test)*float(100)}%")
    print(30*'=')
    if roc:
        probs = classifier.predict_proba(X_test_count_tfidf)
        preds = probs[:,1]
        fpr_svm, tpr_svm, threshold_svm = metrics.roc_curve(y_test, preds)
        roc_auc_svm = metrics.auc(fpr_svm, tpr_svm)

        import matplotlib.pyplot as plt
        plt.title(type(classifier).__name__)
        plt.plot(fpr_svm, tpr_svm, 'b', label = 'AUC = %0.2f' % roc_auc_svm)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()
    
    target_names = ['class 0', 'class 1']
    print(classification_report(y_test, predictions, target_names=target_names))


LINEAR SVM

In [34]:
print("\nLinearSVM with Count Vectors: ")
classifierModel(SVC(kernel='linear', probability=True), X_train_count, y_train, X_test_count,roc=True)

print("\nLinearSVM with Count Vectors + TF-IDF: ")
classifierModel(SVC(kernel='linear', probability=True), X_train_count_tfidf, y_train, X_test_count_tfidf,roc=True)

print("\nLinearSVM with N-Gram Vectors: ")
classifierModel(SVC(kernel='linear', probability=True), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram,roc=False)


LinearSVM with Count Vectors: 


NAIVE BAYES

In [ ]:
print("\nNaive Bayes with Count Vectors: ")
classifierModel(MultinomialNB(), X_train_count, y_train, X_test_count,roc=True)

print("\nNaive Bayes with Count Vectors + TF-IDF: ")
classifierModel(MultinomialNB(), X_train_count_tfidf, y_train, X_test_count_tfidf,roc=True)

print("\nNaive Bayes with N-Gram Vectors: ")
classifierModel(MultinomialNB(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram,roc=False)

RANDOM FOREST

In [ ]:
print("\nRandom Forrest Classifier with Count Vectors: ")
classifierModel(RandomForestClassifier(random_state=rstate,n_estimators=512), X_train_count, y_train, X_test_count,roc=True)

print("\nRandom Forrest Classifier with Count Vectors + TF-IDF: ")
classifierModel(RandomForestClassifier(random_state=rstate,n_estimators=512), X_train_count_tfidf, y_train, X_test_count_tfidf,roc=True)

print("\nRandom Forrest Classifier with N-Gram Vectors: ")
classifierModel(RandomForestClassifier(random_state=rstate,n_estimators=512), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram,roc=False)

KeyboardInterrupt: 

LOGISTIC REGRESSION

In [ ]:
print("\nLogistic Regression with Count Vectors: ")
classifierModel(LogisticRegression(), X_train_count, y_train, X_test_count,roc=True)

print("\nLogistic Regression with Count Vectors + TF-IDF: ")
classifierModel(LogisticRegression(), X_train_count_tfidf, y_train, X_test_count_tfidf,roc=True)

print("\nLogistic Regression with N-Gram Vectors: ")
classifierModel(LogisticRegression(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram,roc=False)